# search jobsdb 

https://hk.jobsdb.com/appium-jobs

In [1]:
!playwright install
!pip install html2text

In [2]:
from playwright.async_api import async_playwright
import os,sys, json, re
import asyncio
import nest_asyncio
from pprint import pprint
import html2text
from IPython.display import display, Markdown, Latex
import random

from SendQuestion import TeamPromptAi
from utils.UrlIgnore import UrlIgnore
from prompts import helloworld, MakeMarkdownString
from utils.log import helloworldLog, logToTextFile

# %run utils/MakeMarkdownString.ipynb

# clear chat history at beginning
%run CleanHistory.ipynb


In [3]:
# with open('./blablabla.txt', 'a+') as f_out:
#     f_out.truncate(0)
#     f_out.writelines(
#         ['1','2','3','4','5']
#     )

In [4]:
temp_general_preprompt = ''
temp_candidate_background = ''
temp_email_preprompt = ''
temp_review_preprompt = ''

BEARER = os.getenv("BEARER",'not found')

with open("./config/general_preprompt.md", "r") as f_general_preprompt:
    temp_general_preprompt = "".join(f_general_preprompt.readlines())

with open("./config/candidate_background.md", "r") as f_candidate_background:
    temp_candidate_background = "".join(f_candidate_background.readlines())

with open("./config/email_preprompt.md", "r") as f_email_preprompt:
    temp_email_preprompt = "".join(f_email_preprompt.readlines())

with open("./config/review_preprompt.md", "r") as f_review_preprompt:
    temp_review_preprompt = "".join(f_review_preprompt.readlines())


In [5]:
async def FetchJobDetail(job):
    job_link = job['job_link']
    print('fetching '+job_link)
    job_details = ''
    
    async with async_playwright() as pw:
        browser = await pw.firefox.launch(headless=True)
        page = await browser.new_page()
        
        await page.goto(job_link)
        title_element = await page.query_selector("title")
        title_text = await title_element.inner_text()

        temp = await page.evaluate("""async () => {
            temp=document.querySelectorAll(`[data-automation~="jobAdDetails"]`)[0].outerHTML
            
            return JSON.stringify({job_details: temp});
        }""")
        job_details = json.loads(temp)['job_details']
        
        await page.screenshot(path="last_screenshot.png", full_page=True)
        await browser.close()   
      
    print('FetchJobDetail done')

    return job_details

In [6]:
async def JobsDBFetchAndGenEmail(kw="appium"):
    job_fetched = []
    async with async_playwright() as pw:
        browser = await pw.firefox.launch(headless=True)
        page = await browser.new_page()
        
        await page.goto("https://hk.jobsdb.com/{kw}-jobs".format(kw=kw))
        
        title_element = await page.query_selector("title")
        title_text = await title_element.inner_text()
        
        await page.screenshot(path="last_screenshot.png", full_page=True)
        
        result = await page.evaluate("""async () => {
            temp=[];
            document.querySelectorAll(`article`).forEach(el => {
                // grab all article elements 
                // article_el = el.parentElement.parentElement  
                try {
                    article_el = el;
                    console.log(article_el);
                    job_title = article_el.querySelector('h3').textContent ;
                    job_company = article_el.querySelector(`[data-automation~="jobCompany"]`).textContent;
                    job_link = article_el.querySelector(`a`).href;
                    temp.push({job_title,job_company, job_link});
                } catch (error) {
                    console.log(el);
                }
            });
            return JSON.stringify(temp);
        }""")
        
        await browser.close()
        
        job_fetched = json.loads(result)
        for job in job_fetched:
            job['meta']={}
            job['meta']['search_using_keyword'] = kw
            job['meta']['source_website'] = 'jobsdb'
    return job_fetched


In [ ]:
def GetEmail(result):
    successful_gen = 0
    
    for job in result:
    
        job_highlight_md = ''
       
        job_meta = job['meta']
        search_using_keyword = job_meta['search_using_keyword']
        source_website = job_meta['source_website']

        job_link = job['job_link']
        job_company = job['job_company']
        job_title = job['job_title']
        m = re.search('.*job/(\d+)\?', job_link)
        job_id = m.group(1)
        
        md_file_name = ''.join([job_id, "_",job_title]).replace(' ','').replace('-','_') + '.md'
        cwd = os.path.abspath("./_output")

        path_with_keyword = '/'.join([
            cwd , 
            search_using_keyword ,
            source_website+'_'+job_id+'_'+job_title.replace('\\','_')
            ]).replace(' ','_').replace("__",'_')
        
        log_path = path_with_keyword
        if not os.path.exists(path_with_keyword):
            os.makedirs(path_with_keyword)

        md_file_path = path_with_keyword + '/' + md_file_name

        url_ignore = UrlIgnore()
        
        try:
            # print(md_file_path)
        
            # print(md_file_path)
    
            # print(job_link)
            # print(url_ignore.check('https://hk.jobsdb.com/job/73509007?type=standout&ref=search-standalone#sol=ad24f6f9405c4b6dfc3e2e7f9def190ba6d7a37b'))
    
            # break
    
            
            if url_ignore.check(job_link.split('?type')[0]):
                # if found in url_ignore then skip
                print("URL: " + job_link + " already in the list, skipping... ")
                continue
        
            job_details = asyncio.run(FetchJobDetail(job))
            
            job_highlight_md = '''
{job_link}

Company Name:{job_company}
Job Title:{job_title}

{job_details}
            '''.format(
                job_link=job_link,
                job_company=job_company,
                job_title=job_title,
                job_details=html2text.html2text(job_details)
            ).strip()
        
            CleanHistory()
            tp_session = TeamPromptAi()
            
            # init bot
            init_bot_result = tp_session.SendQuestion(BEARER,temp_general_preprompt)
            logToTextFile(log_path+'/001_init_log_A.json', json.dumps(init_bot_result))
            print('init bot done')

            # init_bot_result = tp_session.SendQuestion(BEARER,'what are you made for?')
            # init_bot_result['text']
            
            # send job highlight
            question = ('''
Remember the text below as a job highlight and reply "OK"

Text: """
{content}
"""
''').format(content=MakeMarkdownString(job_highlight_md)).strip()
            result = tp_session.SendQuestion(BEARER,question)

            print('send job highlight done')
            logToTextFile(log_path+'/002_job_highlight.md', job_highlight_md)
            logToTextFile(log_path+'/003_send_job_highlight_A.json', json.dumps(result))
            
            # send candidate background
            candidate_background_result = tp_session.SendQuestion(BEARER,
('''
Remember the text below as job-applicant background and reply "OK"

Text: """
{content}
"""
''').format(content=MakeMarkdownString(temp_candidate_background)).strip())
            print('candidate background done')
            logToTextFile(log_path+'/004_candidate_background_A.json', json.dumps(candidate_background_result))
        
            send_email_result = tp_session.SendQuestion(BEARER,
('''
Remember the text below as an email requirement and reply "OK"

Text: """
{content}
"""
''').format(content=MakeMarkdownString(temp_email_preprompt)).strip())
            logToTextFile(log_path+'/005_send_email_A.json', json.dumps(send_email_result))
        
            email_json = tp_session.SendQuestion(BEARER, '''draft me a email''')
            print('draft email done')
            email_content = email_json['text']
            logToTextFile(log_path+'/006_email_content_A.json', json.dumps(email_content))

            # paraphasing, https://www.promptingguide.ai/techniques/tot, tree of thought
            # Then all experts will go on to the next step, etc.
            # If any expert realizes they're wrong at any point, they leave.
            review_result = tp_session.SendQuestion(BEARER, 
('''
{review_preprompt}

Application letter: """
{email_content}
"""
''').format(review_preprompt=temp_review_preprompt, email_content=email_content).strip())
            logToTextFile(log_path+'/007_review_A.json', json.dumps(review_result))
        
            # add to url_ignore
            url_ignore.add(job_link.split('?type')[0])
            print("add url to the list")

            # display(Markdown(email_content))
            # display(Markdown('---'))
            # display(Markdown(review_result['text']))
        
            with open(md_file_path, 'a+') as f_out:
                f_out.truncate(0)
                f_out.writelines([email_content])
                f_out.writelines(['\n\n---\n\n'])
                f_out.writelines([review_result['text']])
        
            # with open('./blablabla.txt', 'a+') as f_out:
            #     f_out.truncate(0)
            #     f_out.writelines([
            #         email_content,'---', review_result['text']
            #         ])

            if (successful_gen > 3):
                break
            else:
                print('gen done')
                successful_gen = successful_gen + 1
            
        
        except Exception as e:
            print("error during processing " + job_link )
            print(e)
            continue
    
    print("done")


: 

In [ ]:
result = []

nest_asyncio.apply()

# javascript
# nodejs
# kw_list = ['IOT','appium','python','react','software validation','software testing','javascript','nodejs']
kw_list = ['IOT']

random.shuffle(kw_list)

for kw in kw_list:
    try:
        temp_result = asyncio.run(JobsDBFetchAndGenEmail(kw))
        # result = result + temp_result
        GetEmail(temp_result)
    
    except Exception as e:
        print("error during search using keyword -> " + kw)
        raise e



fetching https://hk.jobsdb.com/job/73981461?type=standout&ref=search-standalone#sol=0b1221ffc1b3597d7987c14f85f66bb8b788d319
FetchJobDetail done
clean history done
init bot done
send job highlight done
candidate background done
draft email done
store temporary ignored
add url to the list
error during processing https://hk.jobsdb.com/job/73981461?type=standout&ref=search-standalone#sol=0b1221ffc1b3597d7987c14f85f66bb8b788d319
[Errno 2] No such file or directory: '/app/notebook/jobsdb/_output/IOT/jobsdb_73981461_Director_/_Deputy_Director,_Intelligent_IoT_System/73981461_Director/DeputyDirector,IntelligentIoTSystem.md'
fetching https://hk.jobsdb.com/job/73980142?type=standout&ref=search-standalone#sol=ac00d612913c4d8d995e4b9ee383fd3dcec8477a
FetchJobDetail done
clean history done
init bot done
send job highlight done
candidate background done
draft email done
store temporary ignored
add url to the list
error during processing https://hk.jobsdb.com/job/73980142?type=standout&ref=search-st